In [17]:
import torch
import torch.nn as nn
x = torch.randn(1,3,24,24)
x.shape
unfold = nn.Unfold(kernel_size=7, dilation=3, stride=1)
unfold(x).shape

torch.Size([1, 147, 36])

In [13]:
x = torch.randn(1,3,24,24)
x.shape

torch.Size([1, 3, 24, 24])

In [19]:
 torch.einsum("bchw,bchw -> hw",x,x).shape

torch.Size([24, 24])

In [14]:
unfold = nn.Unfold(kernel_size=7, dilation=1, stride=1)
unfold(x).shape

torch.Size([1, 147, 324])

In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class NeighborhoodAttention2D(nn.Module):
    """Neighborhood Attention with Dilation in PyTorch (CUDA Accelerated)."""
    
    def __init__(self, dim, num_heads, kernel_size=7, dilation=1):
        super().__init__()
        self.dim = dim
        self.num_heads = num_heads
        self.kernel_size = kernel_size
        self.dilation = dilation

        # Query, Key, Value transformations
        self.q_proj = nn.Linear(dim, dim, bias=False)
        self.k_proj = nn.Linear(dim, dim, bias=False)
        self.v_proj = nn.Linear(dim, dim, bias=False)
        self.out_proj = nn.Linear(dim, dim, bias=False)

    def forward(self, x):
        """
        x: (B, C, H, W) Input tensor
        Returns: (B, C, H, W) Processed output tensor
        """
        B, C, H, W = x.shape
        q = self.q_proj(x.permute(0, 2, 3, 1))  # (B, H, W, C)
        k = self.k_proj(x.permute(0, 2, 3, 1))
        v = self.v_proj(x.permute(0, 2, 3, 1))

        # Extract local regions with dilation
        unfold = nn.Unfold(kernel_size=self.kernel_size, dilation=self.dilation, stride=1).to(x.device)
        k_unf = unfold(k.permute(0, 3, 1, 2))  # (B, C*K*K, H*W)
        v_unf = unfold(v.permute(0, 3, 1, 2))  # (B, C*K*K, H*W)

        k_unf = k_unf.view(B, C, self.kernel_size * self.kernel_size, H, W)
        v_unf = v_unf.view(B, C, self.kernel_size * self.kernel_size, H, W)

        # Compute attention scores
        attn_scores = torch.einsum("bchw,bcjhw->bjhw", q, k_unf)  # (B, K*K, H, W)
        attn_scores = F.softmax(attn_scores, dim=1)

        # Apply attention to values
        out = torch.einsum("bjhw,bcjhw->bchw", attn_scores, v_unf)

        return self.out_proj(out.permute(0, 3, 1, 2))  # Restore shape (B, C, H, W)



In [6]:
import torch
import torch.nn.functional as F

def extract_dilated_patches(x, kernel_size, dilation):
    """
    Extracts dilated patches from a 4D input tensor using PyTorch.

    Args:
    - x: Input tensor of shape (B, C, H, W)
    - kernel_size: Size of the attention window (e.g., 7 for 7x7)
    - dilation: Dilation factor (e.g., 2, 4, etc.)

    Returns:
    - Extracted patches of shape (B, C, kernel_size*kernel_size, H, W)
    """
    B, C, H, W = x.shape
    padding = (kernel_size // 2) * dilation  # Same padding as in C++ code

    # Unfold extracts sliding windows with dilation
    x_unfolded = F.unfold(x, kernel_size, dilation=dilation, padding=padding)

    # Reshape to (B, C, K*K, H, W)
    num_patches = kernel_size * kernel_size
    x_unfolded = x_unfolded.view(B, C, num_patches, H, W)
    
    return x_unfolded

# Example Usage
B, C, H, W = 1, 3, 32, 32  # Batch, Channels, Height, Width
x = torch.randn(B, C, H, W).cuda()  # Move tensor to CUDA
kernel_size = 7
dilation = 2

patches = extract_dilated_patches(x, kernel_size, dilation)
print("Extracted Patches Shape:", patches.shape)  # Expected: (B, C, K*K, H, W)


Extracted Patches Shape: torch.Size([1, 3, 49, 32, 32])


In [2]:
import torch
import torch.nn as nn
x = torch.randn(1,3,24,24)
x.shape

torch.Size([1, 3, 24, 24])

In [3]:
#Import math Library
import math

#find the  the greatest common divisor of the two integers
print (math.gcd(3, 6))
print (math.gcd(6, 12))
print (math.gcd(12, 36))
print (math.gcd(-12, -36))
print (math.gcd(5, 12))
print (math.gcd(10, 0))
print (math.gcd(0, 34))
print (math.gcd(0, 0))

3
6
12
12
1
10
34
0


In [4]:
class Detect(nn.Module):
    # YOLO Detect head for detection models
    dynamic = False  # force grid reconstruction
    export = False  # export mode
    shape = None
    anchors = torch.empty(0)  # init
    strides = torch.empty(0)  # init

    def __init__(self, nc=80, ch=(), inplace=True):  # detection layer
        super().__init__()
        self.nc = nc  # number of classes